<a href="https://colab.research.google.com/github/gtbnhyujmj/Auto_Shorts_Maker/blob/main/Auto_Paste_PNG2MP4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
# 安裝所需的套件
!pip install opencv-python pillow

In [20]:
import cv2  # OpenCV：處理影片讀寫
from PIL import Image  # Pillow：處理 PNG 圖片，特別是支援透明度
import numpy as np  # 用來做陣列轉換（圖片格式處理）

# ✅ 這個函式負責把 PNG 疊到影片畫面上（會正確處理透明度）

In [21]:
def blend_with_alpha(base_img, overlay_img):
    # 將 OpenCV 讀進來的 BGR 圖片轉成 RGB，再轉成 Pillow 的 RGBA 格式（支援透明）
    base_pil = Image.fromarray(cv2.cvtColor(base_img, cv2.COLOR_BGR2RGB)).convert("RGBA")

    # overlay_img 已經是 PIL.Image 物件，這裡確保它是 RGBA
    overlay_pil = overlay_img.convert("RGBA")

    # 使用 Pillow 的 alpha_composite 把透明圖層貼上去
    blended = Image.alpha_composite(base_pil, overlay_pil)

    # 把合成好的圖轉回 OpenCV 的格式（BGR），這樣才能輸出成影片
    return cv2.cvtColor(np.array(blended.convert("RGB")), cv2.COLOR_RGB2BGR)

# ✅ 主程式：把多張 PNG 疊在 MP4 影片每一幀上

In [22]:
def overlay_multiple_pngs_on_video(video_path, overlay_paths, output_path='output.mp4'):
    # 開啟影片檔案（讀取）
    cap = cv2.VideoCapture(video_path)

    # 讀取影片的參數
    fps = int(cap.get(cv2.CAP_PROP_FPS))  # 每秒幾張幀數（你原始是 30fps）
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))  # 影片寬度
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))  # 影片高度

    # 設定輸出影片格式（mp4v 是支援度較高的格式）
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))  # 建立影片寫出物件

    # 讀入所有 PNG 並先 resize 成跟影片一樣大小
    overlay_images = [Image.open(path).resize((width, height)) for path in overlay_paths]

    frame_idx = 0  # 幀數指標
    while cap.isOpened():  # 當影片還有幀可讀
        ret, frame = cap.read()  # 讀一幀
        if not ret:  # 沒幀了就跳出
            break

        result_frame = frame  # 初始畫面是原始影片的幀

        # 疊加所有圖片
        for overlay_img in overlay_images:
            result_frame = blend_with_alpha(result_frame, overlay_img)

        out.write(result_frame)  # 寫入到輸出影片中
        frame_idx += 1

    cap.release()  # 關掉讀影片的資源
    out.release()  # 關掉寫影片的資源
    print(f"影片儲存至：{output_path}")

# 執行參數區

In [23]:
# 範例使用
video_path = '/content/final_output2.mp4'

overlay_paths = [
    '/content/drive/MyDrive/shorts/characters/char1.png',
    '/content/drive/MyDrive/shorts/PNG_Overlayers/OverLayers2.png',
    # 'folder3/layer3.png',
    # 你可以放任何資料夾的路徑
]

In [24]:
overlay_multiple_pngs_on_video(video_path, overlay_paths, output_path='final_output.mp4')

影片儲存至：final_output.mp4
